# Battle of the Neighbourhoods (Week 2)
### Applied Data Science Capstone (IBM/Coursera)
### Author: Dany Muhajir Sjafiie

## Table of Contents:
* [Introduction/Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussions](#results)
* [Conclusion](#conclusion)

## Introduction/Business Problem <a name="introduction"></a>

## Data <a name="data"></a>

bdjaodancpoq

**Import Necessary Libraries**

In [66]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import numpy as np

import geocoder
import folium 

import json
from pandas.io.json import json_normalize

import requests
from sklearn.cluster import KMeans

import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
%matplotlib inline

import geopandas as gpd

#!pip install xlrd
#!pip install descartes

print("Import Database success!")

Import Database success!


**Import and Process Database**

In [67]:
#Import Table from http://gis.bpbd.jakarta.go.id/
url = "http://gis.bpbd.jakarta.go.id/geoserver/wfs?typename=geonode%3Adki_kelurahan&outputFormat=excel&version=1.0.0&request=GetFeature&service=WFS"
df = pd.read_excel(url)
df = df[["Kecamatan", "KEL_NAME"]]

south_jakarta_subdistrict = ["Kebayoran Baru", "Kebayoran Lama","Pesanggrahan", "Cilandak", "Pasar Minggu", "Jagakarsa", "Mampang Prapatan", "Pancoran", "Tebet", "Setia budi"]
df_all = df.loc[df["Kecamatan"].str.lower().isin([x.lower() for x in south_jakarta_subdistrict])]


df_all= df_all.sort_values(by=["Kecamatan"], ignore_index=True)
df_all.rename(columns={"Kecamatan":"DISTRICT", "KEL_NAME":"URBAN COMMUNITY"}, inplace=True)
df_all.replace({"DISTRICT" : {"SETIA BUDI" : "SETIABUDI"}}, inplace=True)
df_all.replace({"URBAN COMMUNITY" : {"SETIA BUDI" : "SETIABUDI"}}, inplace=True)

#Import Postcode table from 
url_postcode = "https://www.kotajakarta.id/kode-pos-di-kota-jakarta-selatan/"
df_postcode = pd.read_html(url_postcode)

newcolumns = ["POSTCODE","DISTRICT", "URBAN COMMUNITY", "LATITUDE", "LONGITUDE"]
df_all = df_all.reindex(columns=newcolumns, fill_value=0)

for count1,row1 in df_all.iterrows():
    for x in range(len(df_postcode)):
        pc = df_postcode[x]
        for count2,row2 in pc.iterrows():
            if df_all.loc[count1, "URBAN COMMUNITY"].lower() == pc.loc[count2, "Kelurahan"].lower():
                df_all.loc[count1, "POSTCODE"] = pc.loc[count2, "Kode Pos"]


In [68]:
for count1,row1 in df_all.iterrows():
    postcode = df_all.loc[count1, "POSTCODE"]
    urb_com = df_all.loc[count1,"URBAN COMMUNITY"]
    district = df_all.loc[count1,"DISTRICT"]

    g = geocoder.arcgis('{}, {} {}'.format(urb_com,district,postcode))
    df_all.loc[count1,"LATITUDE"] = g.lat
    df_all.loc[count1,"LONGITUDE"] = g.lng

df_all.head()

POSTCODE  DISTRICT   URBAN COMMUNITY  LATITUDE   LONGITUDE
0     12430  CILANDAK    CILANDAK BARAT -6.290702  106.797541
1     12410  CILANDAK    CIPETE SELATAN -6.272805  106.804752
2     12420  CILANDAK  GANDARIA SELATAN -6.270299  106.793017
3     12450  CILANDAK       PONDOK LABU -6.309060  106.797427
4     12440  CILANDAK       LEBAK BULUS -6.300973  106.779093

In [70]:
#Get South Jakarta Coordinate (initially geocoder was used to get coordinate then adjusted to get center of map)
g = geocoder.arcgis("Jakarta Selatan, DKI Jakarta")
latitude = -6.25
longitude = 106.80

#Map South Jakarta
map_southjkt = folium.Map(location=[latitude, longitude], zoom_start=12)

#Import geojson file from http://gis.bpbd.jakarta.go.id and process to only include south jakarta urban communities
southjkt_geo = gpd.read_file('http://gis.bpbd.jakarta.go.id/geoserver/wfs?srsName=EPSG%3A4326&typename=geonode%3Adki_kelurahan&outputFormat=json&version=1.0.0&service=WFS&request=GetFeature')
southjkt_geo = southjkt_geo.loc[southjkt_geo["Kecamatan"].str.lower().isin([x.lower() for x in south_jakarta_subdistrict])]

#folium.Choropleth(
    #geo_data=southjkt_geo,
    #key_on='feature.properties.KEL_NAME',
    #fill_color='deepskyblue').add_to(map_southjkt)

# add markers to map
for lat, lng, label in zip(df_all['LATITUDE'], df_all['LONGITUDE'], df_all['URBAN COMMUNITY']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_southjkt)  
    
map_southjkt

In [122]:
#Define Foursquare Credentials
CLIENT_ID = 'PVUDWQDMYGMZ0LTUGQ0KBGR1YDUDAQWZVAPMRH0MUR5HCMJ2' 
CLIENT_SECRET = '05SPTKSWNBY1IZRO1BD5UFA2ZPGKNJGVUQYI5JLEPILZYNXS' 
VERSION = '20201016' 
LIMIT = 100 

In [123]:
#Define Function to Explore Neighbourhoods
def getNearbyCafes(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Urban Community', 
                  'Urban Community Latitude', 
                  'Urban Community Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    coffee_categories = ["Café", "coffee shop"]
    nearby_cafes = nearby_venues.loc[nearby_venues["Venue Category"].str.lower().isin([x.lower() for x in coffee_categories])]

    return(nearby_cafes)

In [124]:
#Use above function to return venues using toronto_data
southjkt_venues = getNearbyCafes(names=df_all['URBAN COMMUNITY'],
                                   latitudes=df_all['LATITUDE'],
                                   longitudes=df_all['LONGITUDE']
                                  )

CILANDAK BARAT
CIPETE SELATAN
GANDARIA SELATAN
PONDOK LABU
LEBAK BULUS
LENTENG AGUNG
TANJUNG BARAT
SRENGSENG SAWAH
CIGANJUR
CIPEDAK
JAGAKARSA
MELAWAI
PULO
PETOGOGAN
GUNUNG
SENAYAN
GANDARIA UTARA
CIPETE UTARA
RAWA BARAT
SELONG
KRAMAT PELA
GROGOL UTARA
PONDOK PINANG
CIPULIR
KEBAYORAN LAMA SELATAN
KEBAYORAN LAMA UTARA
GROGOL SELATAN
PELA MAMPANG
MAMPANG PRAPATAN
TEGAL PARANG
BANGKA
KUNINGAN BARAT
KALIBATA
CIKOKO
RAWAJATI
PANCORAN
DUREN TIGA
PENGADEGAN
PEJATEN TIMUR
PEJATEN BARAT
JATI PADANG
RAGUNAN
CILANDAK TIMUR
KEBAGUSAN
PASAR MINGGU
PETUKANGAN SELATAN
BINTARO
PETUKANGAN UTARA
PESANGGRAHAN
ULUJAMI
KARET
PASAR MANGGIS
GUNTUR
SETIABUDI
KARET SEMANGGI
MENTENG ATAS
KARET KUNINGAN
KUNINGAN TIMUR
MENTENG DALAM
MANGGARAI SELATAN
MANGGARAI
TEBET BARAT
TEBET TIMUR
BUKIT DURI
KEBON BARU


In [125]:
#Check southjkt_venues database size
print(southjkt_venues.shape)
southjkt_venues.head(20)

(780, 7)


Urban Community  Urban Community Latitude  Urban Community Longitude  \
11  CILANDAK BARAT                 -6.290702                 106.797541   
12  CILANDAK BARAT                 -6.290702                 106.797541   
13  CILANDAK BARAT                 -6.290702                 106.797541   
25  CILANDAK BARAT                 -6.290702                 106.797541   
27  CILANDAK BARAT                 -6.290702                 106.797541   
29  CILANDAK BARAT                 -6.290702                 106.797541   
38  CILANDAK BARAT                 -6.290702                 106.797541   
39  CILANDAK BARAT                 -6.290702                 106.797541   
40  CILANDAK BARAT                 -6.290702                 106.797541   
45  CILANDAK BARAT                 -6.290702                 106.797541   
48  CILANDAK BARAT                 -6.290702                 106.797541   
50  CILANDAK BARAT                 -6.290702                 106.797541   
51  CILANDAK BARAT                 -6.290702                 106.797541   
56  CILANDAK BARAT                 -6.290702                 106.797541   
57  CILANDAK BARAT                 -6.290702                 106.797541   
67  CILANDAK BARAT                 -6.290702                 106.797541   
73  CILANDAK BARAT                 -6.290702                 106.797541   
75  CILANDAK BARAT                 -6.290702                 106.797541   
80  CILANDAK BARAT                 -6.290702                 106.797541   
81  CILANDAK BARAT                 -6.290702                 106.797541   

                       Venue  Venue Latitude  Venue Longitude Venue Category  
11                Bulaf Cafe       -6.287187       106.801288           Café  
12                   Bermvda       -6.291621       106.800385    Coffee Shop  
13                 Starbucks       -6.291604       106.799818    Coffee Shop  
25                 Starbucks       -6.282600       106.791437    Coffee Shop  
27                 Starbucks       -6.291290       106.804650    Coffee Shop  
29         Woodpecker Coffee       -6.284315       106.788571    Coffee Shop  
38              Mars Kitchen       -6.287163       106.795424           Café  
39           Liberica Coffee       -6.291662       106.800271    Coffee Shop  
40                      Tuku       -6.277724       106.801506    Coffee Shop  
45               Toodz House       -6.277903       106.801816           Café  
48  Kedai Tansu (Ketan Susu)       -6.289239       106.802102           Café  
50               PHILOCOFFEE       -6.305373       106.793172    Coffee Shop  
51                 Starbucks       -6.304577       106.795060    Coffee Shop  
56      Jakarta Coffee House       -6.278047       106.804396    Coffee Shop  
57      Crematology x Cipete       -6.277746       106.802556    Coffee Shop  
67         DUA Coffee Cipete       -6.277815       106.800530    Coffee Shop  
73                 Starbucks       -6.283850       106.781015    Coffee Shop  
75              Srsly Coffee       -6.277821       106.805712    Coffee Shop  
80                 Starbucks       -6.294070       106.784957    Coffee Shop  
81                  Gordi HQ       -6.284175       106.811819    Coffee Shop

In [126]:
#Get South Jakarta Coordinate (initially geocoder was used to get coordinate then adjusted to get center of map)
g = geocoder.arcgis("Jakarta Selatan, DKI Jakarta")
latitude = -6.25
longitude = 106.80

#Map South Jakarta
map_southjkt = folium.Map(location=[latitude, longitude], zoom_start=12)

#Import geojson file from http://gis.bpbd.jakarta.go.id and process to only include south jakarta urban communities
southjkt_geo = gpd.read_file('http://gis.bpbd.jakarta.go.id/geoserver/wfs?srsName=EPSG%3A4326&typename=geonode%3Adki_kelurahan&outputFormat=json&version=1.0.0&service=WFS&request=GetFeature')
southjkt_geo = southjkt_geo.loc[southjkt_geo["Kecamatan"].str.lower().isin([x.lower() for x in south_jakarta_subdistrict])]

#folium.Choropleth(
    #geo_data=southjkt_geo,
    #key_on='feature.properties.KEL_NAME',
    #fill_color='deepskyblue').add_to(map_southjkt)

# add markers to map
for lat, lng, label in zip(df_all['LATITUDE'], df_all['LONGITUDE'], df_all['URBAN COMMUNITY']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_southjkt)

for lat, lng, label in zip(southjkt_venues['Venue Latitude'], southjkt_venues['Venue Longitude'], southjkt_venues['Venue']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_southjkt)


    
map_southjkt

## Methodology <a name="methodology"></a>

## Analysis <a name="analysis"></a>

## Results and Discussion <a name="results"></a>

## Conclusion <a name="conclusion"></a>